# N-BEATS Time Series Forecasting
In this tutorial we utilize the N-BEATS (Neural basis expansion analysis for interpretable time series forecasting
) for forecasting the price of ethereum.

For more details regarding N-BEATS, visit this link [https://arxiv.org/abs/1905.10437](https://arxiv.org/abs/1905.10437)

The code for N-BEATS used is adapted from [nbeats-pytorch](https://pypi.org/project/nbeats-pytorch/)

In [ ]:
import pandas as pd
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.nn.functional import mse_loss, l1_loss, binary_cross_entropy, cross_entropy
from torch.optim import Optimizer
import matplotlib.pyplot as plt
import requests
import json
from torch.utils.data import DataLoader, TensorDataset
import numpy as np


In [ ]:
# Fetch data
coins = ["ETH"]
days_ago_to_fetch = 2000 # 7 years
coin_history = {}
hist_length = 0
average_returns = {}
cumulative_returns = {}

def index_history_coin(hist):
    hist = hist.set_index('time')
    hist.index = pd.to_datetime(hist.index, unit='s')
    return hist

def filter_history_by_date(hist):
    result = hist[hist.index.year >= 2017]
    return result

def fetch_history_coin(coin):
    endpoint_url = "https://min-api.cryptocompare.com/data/histoday?fsym={}&tsym=USD&limit={:d}".format(coin, days_ago_to_fetch)
    res = requests.get(endpoint_url)
    hist = pd.DataFrame(json.loads(res.content)['Data'])
    hist = index_history_coin(hist)
    hist = filter_history_by_date(hist)
    return hist

def get_history_from_file(filename):
    return pd.read_csv(filename)


for coin in coins:
    coin_history[coin] = fetch_history_coin(coin)

hist_length = len(coin_history[coins[0]])


In [ ]:
# save to file
# coin_history['ETH'].to_csv("eth_price.csv")

In [ ]:
# read from file
coin_history['ETH'] = get_history_from_file("eth_price.csv")

In [ ]:
# calculate returns

def add_all_returns():
    for coin in coins:
        hist = coin_history[coin]
        hist['return'] = (hist['close'] - hist['open']) / hist['open']
        average = hist["return"].mean()
        average_returns[coin] = average
        cumulative_returns[coin] = (hist["return"] + 1).prod() - 1
        hist['excess_return'] = hist['return'] - average
        coin_history[coin] = hist

add_all_returns()

# display data
cumulative_returns

In [ ]:
# average return per day
average_returns

In [ ]:
# Excess matrix
excess_matrix = np.zeros((hist_length, len(coins)))

for i in range(0, hist_length):
    for idx, coin in enumerate(coins):
        excess_matrix[i][idx] = coin_history[coin].iloc[i]['excess_return']

excess_matrix

In [ ]:
# pretty print excess matrix
pretty_matrix = pd.DataFrame(excess_matrix).copy()
pretty_matrix.columns = coins
pretty_matrix.index = coin_history[coins[0]].index

pretty_matrix

In [ ]:
# Risk modelling

# variance co-var matrix
product_matrix = np.matmul(excess_matrix.transpose(), excess_matrix)
var_covar_matrix = product_matrix / hist_length

var_covar_matrix

In [ ]:
# pretty var_covar
pretty_var_covar = pd.DataFrame(var_covar_matrix).copy()
pretty_var_covar.columns = coins
pretty_var_covar.index = coins

pretty_var_covar

In [ ]:
# Std dev

std_dev = np.zeros((len(coins), 1))
neg_std_dev = np.zeros((len(coins), 1))

for idx, coin in enumerate(coins):
    std_dev[idx][0] = np.std(coin_history[coin]['return'])
    coin_history[coin]['downside_return'] = 0

    coin_history[coin].loc[coin_history[coin]['return'] < 0,
                           'downside_return'] = coin_history[coin]['return']**2
    neg_std_dev[idx][0] = np.sqrt(coin_history[coin]['downside_return'].mean())

In [ ]:
# pretty std
pretty_std = pd.DataFrame(std_dev).copy()
pretty_neg_std = pd.DataFrame(neg_std_dev).copy()
pretty_comb = pd.concat([pretty_std, pretty_neg_std], axis=1)

pretty_comb.columns = ['std dev', 'neg std dev']
pretty_comb.index = coins

pretty_comb

In [ ]:
# std_product_mat
std_product_matrix = np.matmul(std_dev, std_dev.transpose())

# neg_prod_mat
neg_std_product_matrix = np.matmul(neg_std_dev, neg_std_dev.transpose())

In [ ]:
pretty_std_prod = pd.DataFrame(std_product_matrix).copy()
pretty_std_prod.columns = coins
pretty_std_prod.index = coins

pretty_std_prod

In [ ]:
# Corr matrix
corr_matrix = var_covar_matrix / std_product_matrix
pretty_corr = pd.DataFrame(corr_matrix).copy()
pretty_corr.columns = coins
pretty_corr.index = coins

pretty_corr

In [ ]:
# see additional stuff we have added to the DF
coin_history['ETH']

In [ ]:
def simulate_portfolio_growth(initial_amount, daily_returns):
    portfolio_value = [initial_amount]
    for ret in daily_returns:
        portfolio_value.append(portfolio_value[-1] * (1 + ret))
    return portfolio_value

initial_investment = 100000

eth_portfolio = simulate_portfolio_growth(initial_investment, coin_history["ETH"]['return'])

print("ETH Portfolio Growth:", eth_portfolio)

In [ ]:
# Plotting the growth
plt.figure(figsize=(10,6))
plt.plot(eth_portfolio, label='ETH Portfolio', color='blue')
plt.title('Portfolio Growth Over Time')
plt.xlabel('Days')
plt.ylabel('Portfolio Value')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# close dataframe
eth_df = coin_history['ETH'][['close']].copy()

# Convert to tensor
close_tensor = torch.tensor(eth_df.values)

# return dataframe
eth_df = coin_history['ETH'][['return']].copy()

# Convert to tensor
return_tensor = torch.tensor(eth_df.values)

# return_tensor = torch.tensor(eth_df.values)
print(close_tensor)
print(return_tensor)

In [ ]:
# Adapted from https://pypi.org/project/nbeats-pytorch/
# author = {Philippe Remy},

def squeeze_last_dim(tensor):
    if len(tensor.shape) == 3 and tensor.shape[-1] == 1:  # (128, 10, 1) => (128, 10).
        return tensor[..., 0]
    return tensor


def seasonality_model(thetas, t, device):
    p = thetas.size()[-1]
    assert p <= thetas.shape[1], 'thetas_dim is too big.'
    p1, p2 = (p // 2, p // 2) if p % 2 == 0 else (p // 2, p // 2 + 1)
    s1 = torch.tensor(np.array([np.cos(2 * np.pi * i * t) for i in range(p1)])).float()  # H/2-1
    s2 = torch.tensor(np.array([np.sin(2 * np.pi * i * t) for i in range(p2)])).float()
    S = torch.cat([s1, s2])
    return thetas.mm(S.to(device))


def trend_model(thetas, t, device):
    p = thetas.size()[-1]
    assert p <= 4, 'thetas_dim is too big.'
    T = torch.tensor(np.array([t ** i for i in range(p)])).float()
    return thetas.mm(T.to(device))


def linear_space(backcast_length, forecast_length, is_forecast=True):
    horizon = forecast_length if is_forecast else backcast_length
    return np.arange(0, horizon) / horizon

class Block(nn.Module):

    def __init__(self, units, thetas_dim, device, backcast_length=10, forecast_length=5, share_thetas=False,
                 nb_harmonics=None):
        super(Block, self).__init__()
        self.units = units
        self.thetas_dim = thetas_dim
        self.backcast_length = backcast_length
        self.forecast_length = forecast_length
        self.share_thetas = share_thetas
        self.fc1 = nn.Linear(backcast_length, units)
        self.fc2 = nn.Linear(units, units)
        self.fc3 = nn.Linear(units, units)
        self.fc4 = nn.Linear(units, units)
        self.device = device
        self.backcast_linspace = linear_space(backcast_length, forecast_length, is_forecast=False)
        self.forecast_linspace = linear_space(backcast_length, forecast_length, is_forecast=True)
        if share_thetas:
            self.theta_f_fc = self.theta_b_fc = nn.Linear(units, thetas_dim, bias=False)
        else:
            self.theta_b_fc = nn.Linear(units, thetas_dim, bias=False)
            self.theta_f_fc = nn.Linear(units, thetas_dim, bias=False)

    def forward(self, x):
        x = squeeze_last_dim(x)
        x = F.relu(self.fc1(x.to(self.device)))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        return x

    def __str__(self):
        block_type = type(self).__name__
        return f'{block_type}(units={self.units}, thetas_dim={self.thetas_dim}, ' \
               f'backcast_length={self.backcast_length}, forecast_length={self.forecast_length}, ' \
               f'share_thetas={self.share_thetas}) at @{id(self)}'


class SeasonalityBlock(Block):

    def __init__(self, units, thetas_dim, device, backcast_length=10, forecast_length=5, nb_harmonics=None):
        if nb_harmonics:
            super(SeasonalityBlock, self).__init__(units, nb_harmonics, device, backcast_length,
                                                   forecast_length, share_thetas=True)
        else:
            super(SeasonalityBlock, self).__init__(units, forecast_length, device, backcast_length,
                                                   forecast_length, share_thetas=True)

    def forward(self, x):
        x = super(SeasonalityBlock, self).forward(x)
        backcast = seasonality_model(self.theta_b_fc(x), self.backcast_linspace, self.device)
        forecast = seasonality_model(self.theta_f_fc(x), self.forecast_linspace, self.device)
        return backcast, forecast


class TrendBlock(Block):

    def __init__(self, units, thetas_dim, device, backcast_length=10, forecast_length=5, nb_harmonics=None):
        super(TrendBlock, self).__init__(units, thetas_dim, device, backcast_length,
                                         forecast_length, share_thetas=True)

    def forward(self, x):
        x = super(TrendBlock, self).forward(x)
        backcast = trend_model(self.theta_b_fc(x), self.backcast_linspace, self.device)
        forecast = trend_model(self.theta_f_fc(x), self.forecast_linspace, self.device)
        return backcast, forecast



class GenericBlock(Block):

    def __init__(self, units, thetas_dim, device, backcast_length=10, forecast_length=5, nb_harmonics=None):
        super(GenericBlock, self).__init__(units, thetas_dim, device, backcast_length, forecast_length)

        self.backcast_fc = nn.Linear(thetas_dim, backcast_length)
        self.forecast_fc = nn.Linear(thetas_dim, forecast_length)

    def forward(self, x):
        # no constraint for generic arch.
        x = super(GenericBlock, self).forward(x)

        theta_b = self.theta_b_fc(x)
        theta_f = self.theta_f_fc(x)

        backcast = self.backcast_fc(theta_b)  # generic. 3.3.
        forecast = self.forecast_fc(theta_f)  # generic. 3.3.

        return backcast, forecast


class NBEATS(nn.Module):
    SEASONALITY_BLOCK = 'seasonality'
    TREND_BLOCK = 'trend'
    GENERIC_BLOCK = 'generic'

    def __init__(
        self,
        device=torch.device("cpu"),
        stack_types=(GENERIC_BLOCK, GENERIC_BLOCK),
        nb_blocks_per_stack=1,
        forecast_length=7,
        backcast_length=14,
        theta_dims=(2,2),
        share_weights_in_stack=False,
        hidden_layer_units=32,
        nb_harmonics=None,
    ):
        super(NBEATS, self).__init__()
        self.forecast_length = forecast_length
        self.backcast_length = backcast_length
        self.hidden_layer_units = hidden_layer_units
        self.nb_blocks_per_stack = nb_blocks_per_stack
        self.share_weights_in_stack = share_weights_in_stack
        self.nb_harmonics = nb_harmonics  # for seasonal data
        self.stack_types = stack_types
        self.stacks = nn.ModuleList()
        self.thetas_dim = theta_dims
        self.device = device
        print('| N-Beats')
        for stack_id in range(len(self.stack_types)):
            stack = self.create_stack(stack_id)
            self.stacks.append(stack)
        self.to(self.device)
        # self.asset_weight_layer = nn.Softmax(dim=1)
        # self.asset_classes = asset_classes


    def create_stack(self, stack_id):
        stack_type = self.stack_types[stack_id]
        print(f'| --  Stack {stack_type.title()} (#{stack_id}) (share_weights_in_stack={self.share_weights_in_stack})')
        blocks = nn.ModuleList()
        for block_id in range(self.nb_blocks_per_stack):
            block_init = NBEATS.select_block(stack_type)
            if self.share_weights_in_stack and block_id != 0:
                block = blocks[-1]  # pick up the last one when we share weights.
            else:
                block = block_init(
                    self.hidden_layer_units, self.thetas_dim[stack_id],
                    self.device, self.backcast_length, self.forecast_length,
                    self.nb_harmonics
                )
            print(f'     | -- {block}')
            blocks.append(block)
        return blocks

    @staticmethod
    def select_block(block_type):
        if block_type == NBEATS.SEASONALITY_BLOCK:
            return SeasonalityBlock
        elif block_type == NBEATS.TREND_BLOCK:
            return TrendBlock
        else:
            return GenericBlock


    def get_generic_and_interpretable_outputs(self):
        g_pred = sum([a['value'][0] for a in self._intermediary_outputs if 'generic' in a['layer'].lower()])
        i_pred = sum([a['value'][0] for a in self._intermediary_outputs if 'generic' not in a['layer'].lower()])
        outputs = {o['layer']: o['value'][0] for o in self._intermediary_outputs}
        return g_pred, i_pred,

    def forward(self, backcast):
        self._intermediary_outputs = []
        backcast = squeeze_last_dim(backcast)
        forecast = torch.zeros(size=(backcast.size()[0], self.forecast_length,))  # maybe batch size here.
        for stack_id in range(len(self.stacks)):
            for block_id in range(len(self.stacks[stack_id])):
                b, f = self.stacks[stack_id][block_id](backcast)
                backcast = backcast.to(self.device) - b
                forecast = forecast.to(self.device) + f
                block_type = self.stacks[stack_id][block_id].__class__.__name__
                layer_name = f'stack_{stack_id}-{block_type}_{block_id}'

        return backcast, forecast


In [ ]:
from torch.utils.data import Dataset, DataLoader

class TimeSeriesDataset(Dataset):
    def __init__(self, close_data, return_data, backcast_length, forecast_length, shuffle=True):
        self.close_data = close_data
        self.return_data = return_data
        self.backcast_length = backcast_length
        self.forecast_length = forecast_length
        self.indices = list(range(len(self.close_data) - self.backcast_length - self.forecast_length + 1))
        if shuffle:
            np.random.shuffle(self.indices)

    def __len__(self):
        return len(self.close_data) - self.backcast_length - self.forecast_length + 1

    def __getitem__(self, idx):
        start = idx
        end = idx + self.backcast_length
        x = self.close_data[start:end]  # Take columns as needed
        y = self.close_data[end:end+self.forecast_length]  # Adjust as per forecast columns needed
        return x, y

# Hyperparameters
BACKCAST_LENGTH = 14
FORECAST_LENGTH = 7

train_length = round(len(close_tensor) * 0.7)
train_dataset = TimeSeriesDataset(close_tensor[0:train_length], return_tensor[0:train_length], BACKCAST_LENGTH, FORECAST_LENGTH)
test_dataset = TimeSeriesDataset(close_tensor[train_length:], return_tensor[train_length:], BACKCAST_LENGTH, FORECAST_LENGTH)
train_loader = DataLoader(train_dataset)

model = NBEATS(forecast_length=FORECAST_LENGTH, backcast_length=BACKCAST_LENGTH, device=('cuda' if torch.cuda.is_available() else 'cpu'))
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
EPOCHS = 1

num_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of trainable parameters in model: {num_parameters}")

criterion = torch.nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(EPOCHS):
    total_loss = 0.0
    for batch_idx, (x, y) in enumerate(train_loader):
        # Zero gradients
        optimizer.zero_grad()

        x = x.clone().detach().to(dtype=torch.float)
        x = x.to('cuda' if torch.cuda.is_available() else 'cpu')
        y = y.clone().detach().to(dtype=torch.float)
        y = y.to('cuda' if torch.cuda.is_available() else 'cpu')


        # Forward pass
        forecast = model(x)

        loss = criterion(forecast[0], y)

        # Backprop and optimize
        loss.backward()
        optimizer.step()

        # add positive gain for logging
        total_loss += loss  # add the positive gain_loss for logging

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{EPOCHS}, Average Loss: {avg_loss:.4f}")

In [ ]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass

import ezkl
import os
import json

In [ ]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')

witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

In [ ]:
# After training, export to onnx (network.onnx) and create a data file (input.json)
x_export = None
for batch_idx, (x, y) in enumerate(train_loader):
    x_export = x.clone().detach().to(dtype=torch.float)
    break

# Flips the neural net into inference mode
model.eval()

    # Export the model
torch.onnx.export(model,               # model being run
                      x_export,                   # model input (or a tuple for multiple inputs)
                      model_path,            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

    # Serialize data into file:
json.dump( data, open(data_path, 'w' ))

In [ ]:
run_args = ezkl.PyRunArgs()
run_args.input_visibility = "private"
run_args.param_visibility = "fixed"
run_args.output_visibility = "public"
run_args.variables = [("batch_size", 1)]

!RUST_LOG=trace
# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path)
assert res == True

res = ezkl.calibrate_settings(data_path, model_path, settings_path, "resources", max_logrows = 20, scales = [3])
assert res == True

In [ ]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

In [ ]:
# srs path
res = ezkl.get_srs( settings_path)

In [ ]:
res = ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [30]:
res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

spawning module 2


quotient_poly_degree 4
n 262144
extended_k 20


In [ ]:
proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        
        "single",
    )

print(res)
assert os.path.isfile(proof_path)

In [ ]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        
    )

assert res == True
print("verified")